## Python libraries

In [9]:
import networkx as nx
import random
import csv
import pickle
import numpy as np
import os
import spacy
import warnings
import nltk, string
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd 
import json
import math
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from random import randint
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
warnings.filterwarnings('ignore')

## Preprocessing data


In [10]:
def get_train_dict():
    train_data = open('train.txt', 'r') 
    source_list = []
    train_classification_data = []
    nodes_list = []
    source_sink_dict = {}
    while True: 
        sink_list = []
        line = train_data.readline()   
        if not line:    
            break
        nodes = re.split(' |\t',line.strip())
        source = nodes[0]
        for node in nodes[1:]:
            sink_list.append(node)
        source_sink_dict[source] = sink_list
    train_data.close() 
    return source_sink_dict

def get_test_data():
    test_data = pd.read_csv("test-public.csv")
    test_data = test_data.drop(columns=['Id'])
    test_data.columns = ['source', 'sink']
    return test_data

def get_node_features():
    with open('nodes.json') as file:
        data = json.load(file)
    return data

def generate_flatten_source_sink_df():
    source_sink_dict = get_train_dict()
    sink_list = []
    source_list = []
    keys = list(source_sink_dict.keys())

    for key in source_sink_dict.keys():
        sinks = source_sink_dict[key]
        for sink in sinks:
            source_list.append(key)
            sink_list.append(sink)
    source_list = {'Source': source_list, 'Sink': sink_list}  
    
    df_source_sink = pd.DataFrame(source_list) 
    df_source_sink.to_csv('train_source_link_df.csv',header=False,index=False)
    
    g = nx.read_edgelist('train_source_link_df.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print("="*70)
    print('Information about graph')
    print("="*70)
    print(nx.info(g))
    return df_source_sink

def generate_missing_edges():
    generate_flatten_source_sink_df()
    df_source_sink = csv.reader(open('train_source_link_df.csv','r'))
    edges = dict()
    for edge in df_source_sink:
        edges[(edge[0], edge[1])] = 1

    missing_edges = set([])
    while (len(missing_edges) < 53872):
        a= random.randint(0, 4084)
        b= random.randint(0, 4084)
        tmp = edges.get((a,b),-1)
        if tmp == -1 and a!=b:
            try:
                if nx.shortest_path_length(g,source=a,target=b) > 3: 
                    missing_edges.add((a,b))
                else:
                    continue  
            except:  
                missing_edges.add((a,b))              
        else:
            continue
    return missing_edges

def create_dataframes():
    missing_edges = generate_missing_edges()
    print('Length of missing edges:' ,len(missing_edges))
    df_true_relationship = pd.read_csv('train_source_link_df.csv', names=["Source", "Sink"])
    df_false_relationship = pd.DataFrame(list(missing_edges), columns=['Source', 'Sink'])
    frames = [df_true_relationship, df_false_relationship]
    df_relationship = pd.concat(frames)
    true_labels = np.ones(len(df_true_relationship))
    false_labels = np.zeros(len(df_false_relationship))
    labels = np.concatenate((true_labels, false_labels), axis=0)
    print("Number of pair of nodes with edges:", df_true_relationship.shape[0])
    print("Number of pair of nodes without edges:", df_false_relationship.shape[0])
    df_true_relationship.to_csv('df_true_relationship.csv',header=False,index=False)
    print('Total Length:',len(df_relationship))
    return df_relationship, labels
    
def preprocess_keywords(feature):
    keywords_list = [x for x in feature.keys() if x.startswith("keyword")]
    keywords = ' '.join([str(elem) for elem in keywords_list]) 
    return keywords

def preprocess_venues(feature):
    venues_list = [x for x in feature.keys() if x.startswith("venue")]
    venues = ' '.join([str(elem) for elem in venues_list]) 
    return venues

def preprocess_years_publications(feature):
    first_list = [x for x in feature.keys() if x.startswith("first")]
    last_list = [x for x in feature.keys() if x.startswith("last")]
    years_pub = int(feature[first_list[0]]) - int(feature[last_list[0]])
    return years_pub

def preprocess_num_papers(feature):
    num_papers_list = [x for x in feature.keys() if x.startswith("num_papers")]
    return feature[num_papers_list[0]]

def preprocess_first(feature):
    first_list = [x for x in feature.keys() if x.startswith("first")]
    return feature[first_list[0]]

def preprocess_last(feature):
    last_list = [x for x in feature.keys() if x.startswith("last")]
    return feature[last_list[0]]

## Feature Engineering

In [11]:
def showFeatureSelectionGraphs(X_train, clf):
    print("="*100)
    print('Feature Importance')
    print("="*100)
    features = X_train.columns
    importances = clf.feature_importances_
    indices = (np.argsort(importances))[-25:]
    plt.figure(figsize=(6,8))
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Importance')
    plt.show()
    print("="*100)
    print('Correlation Matrix')
    print("="*100)
    plt.figure(figsize=(14,12))
    sns.heatmap(X_train.corr(),
            vmax=0.5,
            square=True,
            annot=True)
    
def calculate_adar(a,b,G):
    sum=0
    try:
        n=list(set(G.successors(a)).intersection(set(G.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(G.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

def jaccard_for_outcoming(a,b, G):
    try:
        if len(set(G.successors(a))) == 0  | len(set(G.successors(b))) == 0:
            return 0
        sim = (len(set(G.successors(a)).intersection(set(G.successors(b)))))/\
                                    (len(set(G.successors(a)).union(set(G.successors(b)))))
    except:
        return 0
    return sim

def jaccard_for_incoming(a,b, G):
    try:
        if len(set(G.predecessors(a))) == 0  | len(set(G.predecessors(b))) == 0:
            return 0
        sim = (len(set(G.predecessors(a)).intersection(set(G.predecessors(b)))))/\
                                 (len(set(G.predecessors(a)).union(set(G.predecessors(b)))))
        return sim
    except:
        return 0
            
def cosine_sim(text1, text2):
    if (text1 == text2):
        return 1
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

def get_features():
    keyword_dict = {}
    venue_dict = {}
    num_papers_dict = {}
    years_publications = {}
    first_dict = {}
    last_dict = {}
    feature_list = get_node_features()
    for feature in feature_list:
        keyword_dict[feature['id']] = preprocess_keywords(feature)
        venue_dict[feature['id']] = preprocess_venues(feature)
        num_papers_dict[feature['id']] = preprocess_num_papers(feature)
        years_publications[feature['id']] = preprocess_years_publications(feature)
        first_dict[feature['id']] = preprocess_first(feature)
        last_dict[feature['id']] = preprocess_last(feature)

    return keyword_dict, venue_dict, num_papers_dict, years_publications,first_dict,last_dict 
     
def assign_features(setX):  
    setX.columns = ['source', 'sink']
    
    G = nx.read_edgelist('df_true_relationship.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)    
    keyword_dict, venue_dict, num_papers_dict, years_publications,first_dict,last_dict = get_features()

    print('Assigning keywords similarity ...')
    setX['keywords_source'] = setX.apply(lambda row: keyword_dict[int(row.source)],axis=1)
    setX['keywords_sink'] = setX.apply(lambda row: keyword_dict[int(row.sink)],axis=1)
    setX['keywords_similarity'] = setX.apply(lambda row: cosine_sim(row.keywords_source, row.keywords_sink) ,axis=1)

    print('Assigning venues similarity ...')
    setX['venues_source'] = setX.apply(lambda row: venue_dict[int(row.source)],axis=1)
    setX['venues_sink'] = setX.apply(lambda row: venue_dict[int(row.sink)],axis=1)
    setX['venues_similarity'] = setX.apply(lambda row: cosine_sim(row.venues_source, row.venues_sink) ,axis=1)

    print('Assigning number of papers ...')
    setX['num_papers_source'] = setX.apply(lambda row: num_papers_dict[int(row.source)],axis=1)
    setX['num_papers_sink'] = setX.apply(lambda row: num_papers_dict[int(row.sink)],axis=1)

    print('Assigning active years of publication by authors ...')
    setX['years_pub_source'] = setX.apply(lambda row: years_publications[int(row.source)],axis=1)
    setX['years_pub_sink'] = setX.apply(lambda row: years_publications[int(row.sink)],axis=1)
    
    print('Assigning in/out degree centrality ...')
    out_degree_centrality = nx.out_degree_centrality(G)
    in_degree_centrality = nx.in_degree_centrality(G)
    setX['source_out_centrality'] = setX.apply(lambda row: out_degree_centrality[int(row.source)] if int(row.source) in out_degree_centrality else 0,axis=1)
    setX['sink_in_centrality'] = setX.apply(lambda row: in_degree_centrality[int(row.sink)] if int(row.sink) in in_degree_centrality else 0,axis=1)
    
    print('Assigning page rank ...')
    page_rank = nx.pagerank_scipy(G)
    setX['source_page_rank'] = setX.apply(lambda row: page_rank[int(row.source)] if int(row.source) in page_rank else 0 ,axis=1)
    setX['sink_page_rank'] = setX.apply(lambda row: page_rank[int(row.sink)] if int(row.sink) in page_rank else 0,axis=1)

    print('Assigning preferential attachment ...')
    setX['preferential_attachment'] = setX.apply(lambda row: row.source_out_centrality * row.sink_in_centrality,axis=1)

    print('Assigning hub and authority score ...')
    hub_score, authority_score = nx.hits(G)
    setX['source_hub_score'] = setX.apply(lambda row: hub_score[int(row.source)] if int(row.source) in hub_score else 0,axis=1)
    setX['sink_hub_score'] = setX.apply(lambda row: hub_score[int(row.sink)] if int(row.sink) in hub_score else 0,axis=1)
    setX['source_authority_score'] = setX.apply(lambda row: authority_score[int(row.source)] if int(row.source) in authority_score else 0,axis=1)
    setX['sink_authority_score'] = setX.apply(lambda row: authority_score[int(row.sink)] if int(row.sink) in authority_score else 0,axis=1)

    print('Assigning katz ...')
    katz = nx.katz.katz_centrality(G,alpha=0.005,beta=1)
    setX['katz_source'] = setX.apply(lambda row: katz[int(row.source)] if int(row.source) in katz else 0, axis =1)
    setX['katz_sink'] = setX.apply(lambda row: katz[int(row.sink)] if int(row.sink) in katz else 0, axis =1)
    
    print('Assigning adar ...')
    setX['adar'] = setX.apply(lambda row: calculate_adar(row.source, row.sink, G) ,axis=1)
    
    print('Assigning jaccard ...')
    setX['jaccard_for_outcoming'] = setX.apply(lambda row: jaccard_for_outcoming(row.source, row.sink, G) ,axis=1)
    setX['jaccard_for_incoming'] = setX.apply(lambda row: jaccard_for_incoming(row.source, row.sink, G) ,axis=1)
    
    print('Assigning number of neighbors ...')
    setX['source_neighbours'] = setX.apply(lambda row: len(list(G.neighbors(int(row.source)))) if int(row.source) in G else 0 ,axis=1)
    setX['sink_neighbours'] = setX.apply(lambda row: len(list(G.neighbors(int(row.sink)))) if int(row.sink) in G else 0 ,axis=1)

    return setX

## Model Selection

In [12]:
def algorithm_comparison(X ,y):
    print("="*100)
    print('Algorithms Comparison')
    print("="*100)
    seed = 7
    models = []
    models.append(('LR', LogisticRegression()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('NB', GaussianNB()))
    models.append(('SVM', SVC()))
    values = []
    names = []
    scoring = 'accuracy'
    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
        values.append(cv_results.mean())
        names.append(name)
        results = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(results)
   
    plt.figure(figsize=(20, 5))
    plt.subplot(131)
    plt.bar(names, values)
    plt.subplot(133)
    plt.plot(names, values)
    plt.suptitle('Accuracy Comparison')
    plt.show()

## Parameter Tuning


In [13]:
def check_best_parameters(X, y):
    print("="*100)
    print('Best parameters RF and LR')
    print("="*100)
    param_dist = {"n_estimators":sp_randint(105,125),
              "max_depth": sp_randint(10,20),
              "min_samples_split": sp_randint(10,100),
              "min_samples_leaf": sp_randint(20,60)}

    clf = RandomForestClassifier(random_state=25,n_jobs=-1)
    rf_random = RandomizedSearchCV(clf, param_distributions=param_dist,n_iter=5,cv=10,scoring='f1',random_state=25)
    rf_random.fit(X,y)
    print('Random Forest')
    print(rf_random.best_params_)
    
    logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200, random_state=0)
    distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])
    clf = RandomizedSearchCV(logistic, distributions, random_state=0)
    search = clf.fit(X, y)
    print('Logistic Regression')
    print(search.best_params_)

## Prediction


In [14]:
def make_prediction(): 
    X, y = create_dataframes()
    print("="*100)
    print('Feature Engineering for training data')
    print("="*100)
    print('Assigning features for train')

    X = assign_features(X)
    X = X.drop(columns=['source','sink','keywords_source', 'keywords_sink','venues_source', 'venues_sink'])

    print('Features were assigned for training data!')

    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=9)
    
    #algorithm_comparison(X, y)
    #check_best_parameters(X_train, y_train)
   
    clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=14, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=23, min_samples_split=11,
            min_weight_fraction_leaf=0.0, n_estimators=121, n_jobs=-1,
            oob_score=False, random_state=25, verbose=0, warm_start=False)
 
    clf.fit(X_train,y_train)
 
    print("="*100)
    print('Results with splitted test data')
    print("="*100)
    y_test_pred = clf.predict(X_test)
    print('Test accuracy',accuracy_score(y_test, y_test_pred))
    print('Test f1 score',f1_score(y_test,y_test_pred))
    
    print("="*100)
    print('Feature Engineering for testing data')
    print("="*100)
    print('Assigning features for test ...')

    submission_test = get_test_data()
    submission_test = assign_features(submission_test)
    submission_test = submission_test.drop(columns=['source','sink','keywords_source', 'keywords_sink','venues_source', 'venues_sink'])
    sub_test_prob = clf.predict_proba(submission_test)
    predictions_df = pd.DataFrame(sub_test_prob)
    predictions_df.to_csv('predictions_prob.csv')
    print('* File for submission created successfully! *')
    
    #showFeatureSelectionGraphs(X_train, clf)
    
make_prediction()

Information about graph
Name: 
Type: DiGraph
Number of nodes: 4016
Number of edges: 53872
Average in degree:  13.4143
Average out degree:  13.4143
Length of missing edges: 53872
Number of pair of nodes with edges: 53872
Number of pair of nodes without edges: 53872
Total Length: 107744
Feature Engineering for training data
Assigning features for train
Assigning keywords similarity ...
Assigning venues similarity ...
Assigning number of papers ...
Assigning active years of publication by authors ...
Assigning in/out degree centrality ...
Assigning page rank ...
Assigning preferential attachment ...
Assigning hub and authority score ...
Assigning katz ...
Assigning adar ...
Assigning jaccard ...
Assigning number of neighbors ...
Features were assigned for training data!
Results with splitted test data
Test accuracy 0.9627824957074574
Test f1 score 0.962721948498652
Feature Engineering for testing data
Assigning features for test ...
Assigning keywords similarity ...
Assigning venues simil